# Excited states methods

Beyond the ground state, the excited states of molecules are often of interest in applications such as light harvesting and bioimaging as well.
In this example, we demonstrate a relatively simple excited state approach, the folded spectrum Hamiltonian (REF!!)

As usual, we start by first defining and building the molecule of interest:

In [1]:
from qibochem.driver import Molecule

h2 = Molecule(xyz_file="../tests/data/h2.xyz")
h2.run_pyscf()

In [2]:
hamiltonian = h2.hamiltonian()
eigenvalues = hamiltonian.eigenvalues()
print("Eigenvalues of the H2 Hamiltonian:")
print(eigenvalues)

[Qibo 0.2.21|INFO|2025-08-11 16:00:55]: Using numpy backend on /CPU:0
[Qibo 0.2.21|WARNING|2025-08-11 16:00:55]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


Eigenvalues of the H2 Hamiltonian:
[-1.13618945 -0.52188556 -0.52188556 -0.47845306 -0.47845306 -0.47845306
 -0.40318375 -0.40318375 -0.1204519   0.30766775  0.30766775  0.44908566
  0.44908566  0.5833141   0.75596744  1.01608717]


The folded spectrum Hamiltonian requires an additional hyperparameter: $\omega$, the number to `fold' the original Hamiltonian about.
Ideally, $\omega$ should be as close to the value of the target excited state as possible.
Here, we'll use the exact ground state, which we will obtain by running VQE normally, as a reference starting point for $\omega$.

In [3]:
import numpy as np

from qibo.models import VQE

from qibochem.ansatz import ucc_ansatz

In [6]:
# We'll start by trying out the UCC circuit ansatz first
ucc_circuit = ucc_ansatz(h2)

# Run VQE to find the ground state for the original molecular Hamiltonian
vqe = VQE(ucc_circuit, hamiltonian)
ucc_parameters = np.random.rand(len(ucc_circuit.get_parameters()))
vqe_ground, ucc_parameters, _result = vqe.minimize(ucc_parameters)
print(f"VQE result: {vqe_ground:.9f}")

VQE result: -1.136188901


Does the VQE result match the lowest eigenvalue from the above cell?
For the small $H_2$ molecule, VQE shouldn't have any problem finding the exact ground state. 

Now, let's try using the (ground state) VQE result in the folded spectrum Hamiltonian.
To try and find the lowest excited state, we will set the hyperparameter $\omega$ to be slightly less negative than the VQE ground state.
Running VQE with this Hamiltonian will yield a new `ground state'.

In [13]:
# Define the folded spectrum Hamiltonian
omega = vqe_ground + 0.5  # Hyperparameter
fs_hamiltonian = h2.fs_hamiltonian(omega)

# Run VQE to find the ground state for the folded spectrum Hamiltonian
fs_vqe = VQE(ucc_circuit, fs_hamiltonian)
fs_parameters = np.random.rand(len(ucc_circuit.get_parameters()))
vqe_fs, fs_parameters, _result = fs_vqe.minimize(fs_parameters)
print(f"VQE folded spectrum result: {vqe_fs:.9f}")

VQE folded spectrum result: 0.024880597


Lastly, the expectation value of the original molecular Hamiltonian is obtained using the circuit parameters that yield the lowest 'energies' for the folded spectrum Hamiltonian.
If everything works, this expectation value should be an excited state of the molecular Hamiltonian.

In [14]:
from qibochem.measurement import expectation

In [17]:
ucc_circuit.set_parameters(fs_parameters)
print(f"Expectation value: {expectation(ucc_circuit, hamiltonian):.8f} (w.r.t. the original Hamiltonian)")

Expectation value: -0.47845306 (w.r.t. the original Hamiltonian)


Comparing this result to the initial set of eigenvalues, we can see that the folded spectum Hamiltonian approach successfully found a higher eigenvalue of the molecular Hamiltonian ($-0.478...$ vs $-1.136...$).

However, one might ask why wasn't the obtained solution the second (degenerate) eigenvalue ($-0.521...$)? The answer lies in the circuit ansatz that was used here.
More specifically, the first two $X$ gates in the UCCSD circuit ansatz define the quantum simulation to have exactly **two** electrons, restricting VQE to searching this particular subspace.
However, the second (degenerate) eigenvalue refers to the case in which there is only **one** electron in the quantum simulation, which is why the default UCC circuit ansatz is unable to locate this solution.

To show this, let's make some slight modifications to the UCCSD circuit ansatz, and run a normal ground state VQE with it.
Essentially, we will just set the initial Hartree-Fock reference circuit (and by extension, the VQE search space) to have only **one** electron.

In [18]:
from qibochem.ansatz import hf_circuit

In [23]:
# Changing the HF reference circuit to have only one electron, and make sure it isn't included in the ucc_ansatz function
ucc_circuit2 = hf_circuit(h2.nso, n_electrons=1) + ucc_ansatz(h2, include_hf=False)

# Run VQE to find the ground state w.r.t. the modified UCCSD circuit ansatz
vqe2 = VQE(ucc_circuit2, hamiltonian)
ucc_parameters2 = np.random.rand(len(ucc_circuit2.get_parameters()))
vqe_ground2, ucc_parameters2, _result = vqe2.minimize(ucc_parameters2)
print(f"VQE result: {vqe_ground2:.9f} (with a one-electron reference HF circuit)")

VQE result: -0.521885562 (with a one-electron reference HF circuit)


So that's an example of how the circuit ansatz can affect the obtained solution in VQE.

Lastly, what happens if we use a circuit ansatz that doesn't conserve the number of electrons in the quantum simulation; *e.g.* the hardware-efficient ansatz?

In [8]:
from qibochem.ansatz import he_circuit

hardware_efficient_circuit = he_circuit(4, 2, coupling_gates="CNOT")
hardware_efficient_circuit.draw()

0: ─RY─RZ─o─────X─RY─RZ─o─────X─
1: ─RY─RZ─X─o───|─RY─RZ─X─o───|─
2: ─RY─RZ───X─o─|─RY─RZ───X─o─|─
3: ─RY─RZ─────X─o─RY─RZ─────X─o─


In [9]:
omega = vqe_ground + 0.3  # Hyperparameter
fs_hamiltonian = h2.fs_hamiltonian(omega)

fs_he_vqe = VQE(hardware_efficient_circuit, fs_hamiltonian)
fs_he_parameters = np.random.rand(len(hardware_efficient_circuit.get_parameters()))
vqe_he_fs, fs_he_parameters, _result = fs_he_vqe.minimize(fs_he_parameters)
print(f"VQE folded spectrum result: {vqe_he_fs:.9f}")

hardware_efficient_circuit.set_parameters(fs_he_parameters)
print("\n", expectation(hardware_efficient_circuit, hamiltonian))

VQE folded spectrum result: 0.098787281

 -0.521884778652012
